In [34]:
from fhirpy import SyncFHIRClient

client = SyncFHIRClient('http://hapi.fhir.org/baseR4') 


In [35]:
practitioner_resource = {
    "resourceType": "Practitioner",
    "name": [{"use": "official", "family": "Ramesh", "given": ["Dr. Rahul"]}]
}

practitioner = client.resource('Practitioner', **practitioner_resource)
practitioner_response = practitioner.save()
practitioner_id = practitioner_response['id']
print(f'Practitioner ID: {practitioner_id}')


Practitioner ID: 45036235


##### Resource: Practitioner
To represent healthcare professionals involved in patient care, enabling proper attribution of medical services and tracking of practitioner-related activities.

- **identifier**: Unique national and institutional IDs.
- **name**: Official name 
- **qualification**: Details of the practitioner's qualifications, including the type of qualification and the issuing organization.
- **active**: Indicator of whether the practitioner is currently active in their role.


In [36]:
patient_resource = {
    "resourceType": "Patient",
    "name": [{"use": "official", "family": "Kumar", "given": ["Arun"]}],
    "gender": "male",
    "birthDate": "2004-09-28",
    "identifier": [{"system": "http://xyzuniversity.org/student-id", "value": "XYZ123456"}],
    "address": [{
        "use": "home",
        "line": ["Room 102, Block C, XYZ University"],
        "city": "Hyderabad",
        "postalCode": "500001"
    }]
}

patient = client.resource('Patient', **patient_resource)
patient_response = patient.save()
patient_id = patient_response['id']
print(f'Patient ID: {patient_id}')


Patient ID: 45036236


##### Resource: Patient
This identifies students and enables us to track the demographic data of students on campus.

- **name**: Student’s full name
- **gender**: [Administrative Gender Value Set](https://hl7.org/fhir/valueset-administrative-gender.html)
- **birthDate**: Date of birth of the student.
- **id**: Unique student identifier like roll no.
- **address**: Student's current residential address

In [37]:
observation_resource = {
    "resourceType": "Observation",
    "status": "final",
    "code": {
        "coding": [{
            "system": "http://loinc.org",
            "code": "LP21258-6",
            "display": "Body temperature"
        }]
    },
    "valueQuantity": {
        "value": 102,
        "unit": "°F",
        "system": "http://unitsofmeasure.org",
        "code": "degF"
    },
    "effectiveDateTime": "2024-09-28T08:00:00Z",
    "subject": {"reference": f'Patient/{patient_id}'}
}

observation = client.resource('Observation', **observation_resource)
observation_response = observation.save()
observation_id = observation_response['id']
print(f'Observation ID: {observation_id}')


Observation ID: 45036237


##### Resource: Observation
To capture symptoms and vital signs recorded by students regularly on campus.

- **code**: [LOINC Observation Codes Value Set](https://build.fhir.org/valueset-observation-codes.html)
- **valueQuantity**: Value of observation (e.g., temperature, blood pressure) [UCUM](https://ucum.org/)
- **effectiveDateTime**: Date and time the observation was recorded.
- **subject**: Reference to the Patient resource.


In [38]:
condition_resource = {
    "resourceType": "Condition",
    "clinicalStatus": {
        "coding": [{
            "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
            "code": "active",
            "display": "Active"
        }]
    },
    "code": {
        "coding": [{
            "system": "http://snomed.info/sct",
            "code": "6142004",
            "display": "Influenza"
        }]
    },
    "onsetDateTime": "2024-09-28T08:00:00Z",
    "subject": {"reference": f'Patient/{patient_id}'}
}

condition = client.resource('Condition', **condition_resource)
condition_response = condition.save()
condition_id = condition_response['id']
print(f'Condition ID: {condition_id}')


Condition ID: 45036238


##### Resource: Condition
To track diagnosed conditions such as flu or COVID-19 for students on campus.

- **clinicalStatus**: [Clinical Status Value Set](https://hl7.org/fhir/valueset-condition-clinical.html)
- **code**: [Condition Code Value Set](https://build.fhir.org/valueset-condition-code.html)
- **onsetDateTime**: Date and time when the condition was first identified.
- **subject**: Reference to the Patient resource.


In [39]:
appointment_resource = {
    "resourceType": "Appointment",
    "status": "booked",
    "start": "2024-09-28T10:00:00Z",
    "end": "2024-09-28T10:30:00Z",
    "participant": [
        {
            "actor": {"reference": f'Patient/{patient_id}'},
            "status": "accepted"
        },
        {
            "actor": {"reference": f'Practitioner/{practitioner_id}'},
            "status": "accepted"
        }
    ]
}

appointment = client.resource('Appointment', **appointment_resource)
appointment_response = appointment.save()
appointment_id = appointment_response['id']
print(f'Appointment ID: {appointment_id}')


Appointment ID: 45036239


##### Resource: Appointment
To manage healthcare appointments between students and campus healthcare providers.

- **start**: Start time of the appointment.
- **end**: End time of the appointment.
- **participant**: List of participants [Participant Type Value Set](http://hl7.org/fhir/ValueSet/participant-type)
- **status**: (e.g., booked, cancelled) [Appointment Status Value Set](http://hl7.org/fhir/ValueSet/appointmentstatus)


In [40]:
document_reference_resource = {
    "resourceType": "DocumentReference",
    "status": "current",
    "subject": {
        "reference": f"Patient/{patient_id}"
    },
    "type": {
        "coding": [
            {
                "system": "http://loinc.org",
                "code": "34133-9",
                "display": "Summary of care plan"
            }
        ]
    },
    "content": [
        {
            "attachment": {
                "contentType": "application/pdf",
                "url": "urn:uuid:careplan-flu-treatment"  
            }
        }
    ]
}

document_reference = client.resource('DocumentReference', **document_reference_resource)
document_reference_response = document_reference.save()
document_reference_id = document_reference_response['id']
print(f'DocumentReference ID: {document_reference_id}')


DocumentReference ID: 45036240


##### Resource: DocumentReference
To store and reference documents related to the student's health (e.g., lab reports, prescriptions, vaccination certificates).

- **type**: (e.g., lab report, prescription) [Document Type Codes Value Set](https://build.fhir.org/valueset-doc-typecodes.html)
- **content**: The actual document, usually in binary format (PDF, image) [HL7 Format Codes Value Set](https://terminology.hl7.org/6.0.2/ValueSet-v3-HL7FormatCodes.html)
- **subject**: Reference to the Patient resource.


In [41]:
medication_request_resource = {
    "resourceType": "MedicationRequest",
    "status": "active",
    "intent": "order",
    "medicationCodeableConcept": {
        "coding": [{
            "system": "http://snomed.info/sct",
            "code": "324337001",
            "display": "Oseltamivir 75 MG Oral Capsule"
        }]
    },
    "subject": {"reference": f'Patient/{patient_id}'},
    "authoredOn": "2024-09-28T09:00:00Z",
    "dosageInstruction": [{
        "text": "Take 75mg twice daily for 5 days",
        "timing": {
            "repeat": {
                "frequency": 2,
                "period": 1,
                "periodUnit": "d"
            }
        },
        "doseAndRate": [{
            "doseQuantity": {
                "value": 75,
                "unit": "mg",
                "system": "http://unitsofmeasure.org",
                "code": "mg"
            }
        }]
    }]
}

medication_request = client.resource('MedicationRequest', **medication_request_resource)
medication_request_response = medication_request.save()
medication_request_id = medication_request_response['id']
print(f'Medication Request ID: {medication_request_id}')


Medication Request ID: 45036241


#### Resource: MedicationRequest
To store prescribed medications for students, enabling medication tracking.
- **medication**: [Medication Codes Value Set - FHIR v6.0.0-ballot1](https://hl7.org/fhir/ValueSet/medication-codes.html)
- **dosageInstruction**: Specific instructions [Timing Abbreviation Value Set](http://hl7.org/fhir/ValueSet/timing-abbreviation)
- **subject**: Reference to the Patient resource.
- **authoredOn**: Date the medication was prescribed.


In [42]:
communication_resource = {
    "resourceType": "Communication",
    "status": "completed",
    "subject": {"reference": f'Patient/{patient_id}'},
    "topic": {
        "coding": [{
            "system": "http://terminology.hl7.org/CodeSystem/communication-topic",
            "code": "flu-prevention",
            "display": "Flu Prevention Alert"
        }]
    },
    "sent": "2024-09-28T10:05:00Z",
    "payload": [{
        "contentString": "Alert: Please wear masks and take precautions as flu cases have increased on campus."
    }]
}

communication = client.resource('Communication', **communication_resource)
communication_response = communication.save()
communication_id = communication_response['id']
print(f'Communication ID: {communication_id}')


Communication ID: 45036242


##### Resource: Communication
To send alerts to students and healthcare providers regarding potential health outbreaks.
- **status**: [Event Status Value Set](https://build.fhir.org/valueset-event-status.html)
- **subject**: Reference to the Patient resource.
- **topic**: [Communication Topic Value Set](https://build.fhir.org/valueset-communication-topic.html)
- **sent**: Date the communication was sent.


In [43]:
medication_administration_resource = {
    "resourceType": "MedicationAdministration",
    "status": "completed",
    "medicationCodeableConcept": {
        "coding": [{
            "system": "http://snomed.info/sct",
            "code": "324337001",
            "display": "Oseltamivir 75 MG Oral Capsule"
        }]
    },
    "subject": {"reference": f'Patient/{patient_id}'},
    "effectiveDateTime": "2024-09-28T12:00:00Z",
    "dosage": {
        "dose": {
            "value": 75,
            "unit": "mg",
            "system": "http://unitsofmeasure.org",
            "code": "mg"
        }
    }
}

medication_administration = client.resource('MedicationAdministration', **medication_administration_resource)
medication_administration_response = medication_administration.save()
medication_administration_id = medication_administration_response['id']
print(f'Medication Administration ID: {medication_administration_id}')


Medication Administration ID: 45036243


##### Resource: MedicationAdministration
To record the administration of medications to students, facilitating medication tracking.

- **medication**: What was administered [Medication Codes Value Set](https://build.fhir.org/valueset-medication-codes.html)
- **subject**: Reference to the Patient resource.
- **occurrence**: When the medication was administered.
- **dosage**: Dosage details including amount and timing [UCUM](https://ucum.org/)


In [44]:
care_plan_resource = {
    "resourceType": "CarePlan",
    "status": "active",
    "intent": "plan",
    "title": "Flu Treatment Care Plan",
    "subject": {"reference": f'Patient/{patient_id}'},
    "activity": [{
        "detail": {
            "code": {
                "coding": [{
                    "system": "http://snomed.info/sct",
                    "code": "76445007",
                    "display": "Treatment regimen"
                }]
            },
            "status": "in-progress",
            "prohibited": False
        }
    }]
}

care_plan = client.resource('CarePlan', **care_plan_resource)
care_plan_response = care_plan.save()
care_plan_id = care_plan_response['id']
print(f'Care Plan ID: {care_plan_id}')


Care Plan ID: 45036244


##### Resource: CarePlan
To manage personalized care plans for students (including preventive, treatment, post-recovery).

- **status**: Current status of the care plan. [Request Status Value Set](https://build.fhir.org/valueset-request-status.html)
- **intent**: Plan/Proposal, etc. [Care Plan Intent Value Set](https://build.fhir.org/valueset-care-plan-intent.html)
- **activity**: Specific activities [Care Plan Activity Performed Value Set](https://build.fhir.org/valueset-care-plan-activity-performed.html)
- **subject**: Reference to the Patient resource.


In [45]:
def get_resource_by_id(resource_type, resource_id):
    resource = client.resources(resource_type).search(_id=resource_id).first() 
    return resource  

def print_resource_details(resource):
    if resource:
        print()
        print(f"Details of {resource.resourceType}:")
        for key, value in resource.items():
            print(f"{key}: {value}")
        print()
    else:
        print("Resource not found.")


In [46]:
retrieved_practitioner = get_resource_by_id('Practitioner', practitioner_id)
print_resource_details(retrieved_practitioner)

retrieved_patient = get_resource_by_id('Patient', patient_id)
print_resource_details(retrieved_patient)

retrieved_observation = get_resource_by_id('Observation', observation_id)
print_resource_details(retrieved_observation)

retrieved_appointment = get_resource_by_id('Appointment', appointment_id)
print_resource_details(retrieved_appointment)

retrieved_condition = get_resource_by_id('Condition', condition_id)
print_resource_details(retrieved_condition)

retrieved_medication_request = get_resource_by_id('MedicationRequest', medication_request_id)
print_resource_details(retrieved_medication_request)

retrieved_medication_administration = get_resource_by_id('MedicationAdministration', medication_administration_id)
print_resource_details(retrieved_medication_administration)

retrieved_careplan = get_resource_by_id('CarePlan', care_plan_id)
print_resource_details(retrieved_careplan)

retrieved_communication = get_resource_by_id('Communication', communication_id)
print_resource_details(retrieved_communication)


Details of Practitioner:
resourceType: Practitioner
id: 45036235
meta: {'versionId': '1', 'lastUpdated': '2024-09-28T13:25:23.673+00:00', 'source': '#z10fttI2qsbL2Kr2'}
name: [{'use': 'official', 'family': 'Ramesh', 'given': ['Dr. Rahul']}]


Details of Patient:
resourceType: Patient
id: 45036236
meta: {'versionId': '1', 'lastUpdated': '2024-09-28T13:25:24.674+00:00', 'source': '#h7xNxqVSOTG5c1mH'}
text: {'status': 'generated', 'div': '<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText">Arun <b>KUMAR </b></div><table class="hapiPropertyTable"><tbody><tr><td>Identifier</td><td>XYZ123456</td></tr><tr><td>Address</td><td><span>Room 102, Block C, XYZ University </span><br/><span>Hyderabad </span></td></tr><tr><td>Date of birth</td><td><span>28 September 2004</span></td></tr></tbody></table></div>'}
identifier: [{'system': 'http://xyzuniversity.org/student-id', 'value': 'XYZ123456'}]
name: [{'use': 'official', 'family': 'Kumar', 'given': ['Arun']}]
gender: male
birthDate: